# Siamese Network



---

In this session, we are going to implement a Siamese Network.

It takes as input two augmented versions of the same image and produces as output two feature vectors one for each version of the image.

For simplicity, we will use the same backbone to process the views as in SimCLR paper.

Andiamo a costruire una Siamese Architecture. Abbiamo un encoder che processa allo stesso tempo sia una immagine augmented che l'altra versione. Per ogni forward di una SN si va a fare due forward dell'encoder: uno che processa l'immagine e uno che processa l'altra.

Si prende come backbone una resnet18 e andare a costruire una siamese network. Vogliamo implementarne il costruttore e il forward. Il costruttore dovrà inizializzare due versioni dell'encoder e il forward, prese x1 e x2 che sono le view1 e view2, restituisce in uscita le feature di queste immagini.

(il costruttore dovrà inizializzare due versione dell'encoder, (le view sono x1 e x2) e devono fare il forward di queste immagini e devono restituire in uscita...)

Ci fermiamo alle y lasciando stare l'MLP. Quindi prendiamo le view xi e xj, le processo con f() e ottengo yi e yj (negli appunti hi e hj).


In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.models as models
import torchvision.transforms as transforms



import torchvision
import torchvision.models as models
#import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import pandas as pd
from torchvision.io import read_image



from torch.utils.data import Dataset
from torchvision import datasets, transforms
import torchvision.models as models
import torchvision.transforms as transforms

In [2]:
# you can use a resnet18 as backbone
backbone = models.resnet18()
#print(backbone)

#! remember to delete the fc layer (we need just the CNN layers + flatten)
backbone.fc = nn.Identity() # crea un fc finto che fa Identity(), la backbone così non avrà più come uscita un linear layer (che è fc), bensì avrà un layer identità che non fa nulla

# Identity() prende x e ritorna x, non fa nulla! Lo facciamo perché sennò si rompe il forward

print(backbone)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
class CustomImageDataset(Dataset):
    def __init__(self, data, targets = None, transform=None, target_transform=None): # valori di default
        self.imgs = data
        self.targets = targets
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img = self.imgs[idx]
        if isinstance(img, str):
          image = read_image(img)
        else:
          image = Image.fromarray(img.astype('uint8'), 'RGB')
        if self.transform: # arriva qui con una PIL image
            image1 = self.transform(image) # fa due trasformazioni
            image2 = self.transform(image)
        else:
            image1 = image
            image2 = image
        return image1, image2

Abbiamo due versioni di Siamese Network. 
1) c'è una simmetria della rete, lo stesso encoder è utilizzato in tutti i due branch della SN. 
2) c'è asimmetria tra le due reti, si hanno due encoder diversi (encoder1 con una rete ed encoder2 con un'altra rete) - si duplica il codice -

Vediamo la versione più generale con due encoder diversi tra i due branch della rete.

In [8]:
class Identity(torch.nn.Module):
  def forward(self, x):
    return x


class SiameseNet(nn.Module):
    def __init__(self, backbone): # si prende in input la backbone (può essere la ResNet18 per esempio) e istanzio due versioni dell'encoder (encoder1 ed encoder2)
        super().__init__()
        self.encoder1 = backbone
        # replace the fc layer with the identity layer
        self.encoder1.fc = Identity() # equal to nn.Identity() # poi si fa il replace del fc con Identity()

        self.encoder2 = backbone
        self.encoder2.fc = Identity() # equal to nn.Identity()

        # or, versione con un encoder
        # self.encoder = backbone
        # self.encoder.fc = Identity() # equal to nn.Identity()


    def forward(self, x1, x2, return_dict = True): # il forward prende in input le due view. qui si chiama il forward dell'encoder1 e dell'encoder2
        x1 = self.encoder1(x1)
        x2 = self.encoder2(x2)
        if return_dict:
          return {'view1': x1, 'view2': x2,} # in input ho le immagini, in uscita avrà le feature estratte con l'encoder1 e l'encoder2

        # versione con un solo encoder
        # x1 = self.encoder(x1)
        # x2 = self.encoder(x2)

        # return x1, x2 #


        # return {x1, x2} lista
        # return torch.cat((x1, x2), dim = 0) # most preferred one, tensore con versioni augmented sono concatenate in verticale




# Check output shape
#x1, x2 = SiameseNet(backbone)(torch.randn(5, 3, 32, 32), torch.randn(5, 3, 32, 32)).shape





Let's now use the Dataset which creates the two augmented views for each image from the [past lab session](https://colab.research.google.com/drive/1NJwAFbRiD4MdwWf__6P2Lm0xYk_DNdVu?usp=sharing) and create a loop with forward pass

In [ ]:
#trainset = ... # scorsa volta, due uscite

data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True) # dataset

color_jitter = transforms.ColorJitter(0.8, 0.8, 0.8, 0.2) # si distorce il colore con una certa probabilità tutti i canali


transform = transforms.Compose([transforms.RandomResizedCrop(size=32),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.RandomApply([color_jitter], p=0.8),
                                  transforms.RandomGrayscale(p=0.2),
                                  transforms.GaussianBlur(kernel_size=int(0.1 * 32)),
                                  transforms.ToTensor()])

trainset = CustomImageDataset(data.data, transform = transform)
dataloader = DataLoader(trainset, batch_size=64, shuffle=True)

model = SiameseNet(backbone)

for idx, data in enumerate(dataloader):
    view1, view2 = data
    output = model(view1, view2)
    print(f"batch {idx}:")
    print(output['view1'].shape)
    print(output['view2'].shape)
    print()

    if idx == 3:
        break